In [ ]:
# See how much memory is used by OS initially.
# Then substract this value from later mem used, to obtain mem used by scripts.
import psutil
GB_UNIT = 1000*1024**2
MB_UNIT = GB_UNIT/1e3
OS_MEM = psutil.virtual_memory().used
print(OS_MEM/GB_UNIT)

In [ ]:
from shared.preface import *
import shared.functions as fct

# Initialize parameters and files.
PRE = PRE(
    sim='L025N752', 
    z0_snap=36, z4_snap=13, DM_lim=10000,
    sim_dir=SIM_ROOT, sim_ver=SIM_TYPE,
    phis=20, thetas=20, vels=200,
    pre_CPUs=128, sim_CPUs=128
)


TEMP_DIR = f'X_tests'

Testing = False
if Testing:
    mass_gauge = 12.3
    mass_range = 0.3
    size = 1
else:
    mass_gauge = 12.0
    mass_range = 0.6
    size = 10

hname = f'1e+{mass_gauge}_pm{mass_range}Msun'
fct.halo_batch_indices(
    PRE.Z0_STR, mass_gauge, mass_range, 'halos', size, 
    hname, PRE.SIM_DIR, TEMP_DIR
)
halo_batch_IDs = np.load(f'{TEMP_DIR}/halo_batch_{hname}_indices.npy')
halo_batch_params = np.load(f'{TEMP_DIR}/halo_batch_{hname}_params.npy')
halo_num = len(halo_batch_params)

print('********Number density band********')
print('Halo batch params (Rvir,Mvir,cNFW):')
print(halo_batch_params)
print('***********************************')

In [ ]:
# =============================================== #
# Run precalculations for selected halo in batch. #
# =============================================== #

halo_j = 2
halo_ID = halo_batch_IDs[halo_j]

# Generate progenitor index array for current halo.
splits = re.split('/', SIM_TYPE)
MTname = f'{PRE.SIM}_{splits[0]}_{splits[1]}'
proj_IDs = fct.read_MergerTree(PRE.OUT_DIR, MTname, halo_ID)

# Create empty arrays to save specifics of each loop.
save_GRID_L = np.zeros(len(PRE.NUMS_SNAPS))
save_num_DM = np.zeros(len(PRE.NUMS_SNAPS))
save_DM_com = []

# for j, (snap, proj_ID) in enumerate(zip(
#     PRE.NUMS_SNAPS[::-1], proj_IDs
# )):

j = 0
snap = PRE.NUMS_SNAPS[::-1][0]
proj_ID = proj_IDs[0]

proj_ID = int(proj_ID)

# Output halo progress.
print(f'halo {halo_j+1}/{halo_num} ; snapshot {snap}')


# --------------------------- #
# Read and load DM positions. #
# --------------------------- #

# Define how many shells are used, out of len(DM_SHELL_EDGES)-1.
shells = 4
DM_shell_edges = DM_SHELL_EDGES[:shells+1]

IDname = f'origID{halo_ID}_snap_{snap}'
fct.read_DM_all_inRange(
    snap, proj_ID, DM_shell_edges, IDname, PRE.SIM_DIR, TEMP_DIR
)

# Load DM from all used shells.
DM_pre = []
for si in range(shells):
    DM_pre.append(np.load(f'{TEMP_DIR}/DM_pos_{IDname}_shell{si}.npy'))
DM_raw = np.array(list(chain.from_iterable(DM_pre)))
del DM_pre

# Save c.o.m. coord of all DM particles (for outside_gravity fct.).
save_DM_com.append(np.sum(DM_raw, axis=0)/len(DM_raw))

# ---------------------- #
# Cell division process. #
# ---------------------- #

# Initialize grid.
snap_GRID_L = (int(np.abs(DM_raw).max()) + 1)*kpc
raw_grid = fct.grid_3D(snap_GRID_L, snap_GRID_L)
init_grid = np.expand_dims(raw_grid, axis=1)

# Prepare arrays for cell division.
DM_raw *= kpc
DM_pos = np.expand_dims(DM_raw, axis=0)
DM_pos_for_cell_division = np.repeat(DM_pos, len(init_grid), axis=0)

# Cell division.
cell_division_count = fct.cell_division(
    init_grid, DM_pos_for_cell_division, snap_GRID_L, PRE.DM_LIM, None, TEMP_DIR, IDname
)

In [ ]:
8*(100000*3)/1.e6

In [ ]:
def cell_gravity_short_range(
    cell_coords_in, cell_gen, init_GRID_S,
    DM_pos, DM_lim, DM_sim_mass, smooth_l,
    out_dir, b_id
):

    cell_coords = np.expand_dims(np.array(cell_coords_in), axis=1)
    cell_gen = np.array(cell_gen)

    # Center all DM positions w.r.t. cell center.
    DM_pos_sync = np.repeat(DM_pos, len(cell_coords), axis=0)
    DM_pos_sync -= cell_coords

    # Cell lengths to limit DM particles. Limit for the largest cell is 
    # GRID_S/2, not just GRID_S, therefore the cell_gen+1 !
    cell_len = np.expand_dims(init_GRID_S/(2**(np.array(cell_gen)+1)), axis=1)

    # Select DM particles inside each cell based on cube length generation.
    DM_in_cell_IDs = np.asarray(
        (np.abs(DM_pos_sync[:,:,0]) < cell_len) & 
        (np.abs(DM_pos_sync[:,:,1]) < cell_len) & 
        (np.abs(DM_pos_sync[:,:,2]) < cell_len)
    )
    del cell_gen, cell_len

    # Set DM outside cell to nan values.
    DM_pos_sync[~DM_in_cell_IDs] = np.nan
    del DM_in_cell_IDs

    # Sort all nan values to the bottom of axis 1, i.e. the DM-in-cell-X axis 
    # and truncate array based on DM_lim parameter. This simple way works since 
    # each cell cannot have more than DM_lim.
    ind_2D = DM_pos_sync[:,:,0].argsort(axis=1)
    ind_3D = np.repeat(np.expand_dims(ind_2D, axis=2), 3, axis=2)
    DM_sort = np.take_along_axis(DM_pos_sync, ind_3D, axis=1)
    DM_in = DM_sort[:,:DM_lim*SHELL_MULTIPLIERS[-1],:]

    
    # note: Memory peaks here, due to these arrays:
    # print(DM_pos_sync.shape, ind_2D.shape, ind_3D.shape, DM_sort.shape, DM_in.shape)
    # mem_inc = gso(cell_coords)+gso(DM_pos_sync)+gso(ind_2D)+gso(ind_3D)+gso(DM_sort)+gso(DM_in)
    # print('MEM_PEAK:', mem_inc/1e6)
    del DM_pos_sync, ind_2D, ind_3D, DM_sort

    # Calculate distances of DM and adjust array dimensionally.
    DM_dis = np.expand_dims(np.sqrt(np.sum(DM_in**2, axis=2)), axis=2)

    # Offset DM positions by smoothening length of Camila's simulations.
    eps = smooth_l / 2.

    # nan values to 0 for numerator, and 1 for denominator to avoid infinities.
    quot = np.nan_to_num(cell_coords - DM_in, copy=False, nan=0.0) / \
        np.nan_to_num(
            np.power((DM_dis**2 + eps**2), 3./2.), copy=False, nan=1.0
        )
    
    # note: Minus sign, s.t. velocity changes correctly (see GoodNotes).
    derivative = -G*DM_sim_mass*np.sum(quot, axis=1)    
    np.save(f'{out_dir}/batch{b_id}_short_range.npy', derivative)


def cell_gravity_long_range(
    c_id, b_id, cellX_coords, 
    DM_count, cell_com, 
    DM_sim_mass, smooth_l, out_dir
):

    # Distances between cell centers and cell c.o.m. coords.
    com_dis = np.expand_dims(
        np.sqrt(np.sum((cellX_coords-cell_com)**2, axis=1)), axis=1
    )

    # Offset DM positions by smoothening length of Camila's simulations.
    eps = smooth_l / 2.

    # Long-range gravity component for each cell (without including itself).
    quot = (cellX_coords-cell_com)/np.power((com_dis**2 + eps**2), 3./2.)
    DM_count_sync = np.expand_dims(DM_count, axis=1)
    del com_dis

    # note: Minus sign, s.t. velocity changes correctly (see GoodNotes).
    derivative = -G*DM_sim_mass*np.sum(DM_count_sync*quot, axis=0)
    
    # note: Memory peaks here, due to these arrays:
    # print(quot.shape, DM_count_sync.shape, derivative.shape)
    # mem_inc = gso(quot)+gso(DM_count_sync)+gso(derivative)
    # print(mem_inc/1e6)
    del quot, DM_count_sync

    np.save(f'{out_dir}/cell{c_id}_batch{b_id}_long_range.npy', derivative)


def chunksize_long_range(cells, core_mem_MB):
    
    # note: mem_MB specific to peak memory usage in cell_gravity_long_range.
    # -> Peak memory after calculation of derivative.

    elem = 8                          # 8 bytes for standard np.float64
    mem_type1 = 3*elem                # for derivative
    mem_type2 = cells*3*elem          # for quot
    mem_type3 = cells*elem            # for DM_count_sync

    mem_MB = (mem_type1+mem_type2+mem_type3)/1.e6

    batches = 1
    while mem_MB >= 0.95*core_mem_MB:
        mem_MB *= batches
        batches += 1
        mem_MB /= batches

    chunksize = math.ceil(cells/batches)

    return chunksize


def chunksize_short_range(cells, DM_tot, max_DM_lim, core_mem_MB):

    # note: mem_MB specific to peak memory usage in cell_gravity_short_range.
    # -> Peak memory after calculation of ind_2D,ind_3D,etc. sorting arrays.

    elem = 8                               # 8 bytes for standard np.float64
    mem_type0 = cells*3 * elem             # for list to ndarray of cell_coords
    mem_type1 = cells*DM_tot * elem        # for ind_2D
    mem_type2 = cells*DM_tot*3 * elem      # for DM_pos_sync, ind_3D, DM_sort
    mem_type3 = cells*max_DM_lim*3 * elem  # for DM_in

    mem_MB = (mem_type0+mem_type1+(3*mem_type2)+mem_type3)/1.e6

    batches = 1
    while mem_MB >= 0.95*core_mem_MB:
        mem_MB *= batches
        batches += 1
        mem_MB /= batches

    chunksize = math.ceil(cells/batches)

    return chunksize


def batch_generators_short_range(cell_coords, cell_gen, chunksize):

    cells = len(cell_coords)

    batches = math.ceil(cells/chunksize)
    print(f'Nr. of batches (short-range): {batches}')
    batch_arr = np.arange(batches)

    cell_chunks = chunks(chunksize, cell_coords)
    cgen_chunks = chunks(chunksize, cell_gen)
    
    return batch_arr, cell_chunks, cgen_chunks


def batch_generators_long_range(
    cell_coords, com_coords, DM_counts,
    chunksize 
):
    cells = len(cell_coords)
    cell_nums = np.arange(cells)

    batches = math.ceil(cells/chunksize)
    print(f'Nr. of batches (long-range): {batches}')

    # Arrays used for naming files.
    id_arr = np.array([idx+1 for idx in cell_nums for _ in range(batches)])
    batch_arr = np.array([b+1 for _ in cell_nums for b in range(batches)])

    # Coord of cell, for which long-range gravity gets calculated.
    coord_arr = np.array([cc for cc in cell_coords for _ in range(batches)])

    # Chunks for DM_count array, as a generator for all cells.
    count_gens = (c for _ in cell_nums for c in chunks(chunksize, DM_counts))
    count_chain = chain(gen for gen in count_gens)

    # Chunks for cell_com array, as a generator for all cells.
    com_gens = (c for _ in cell_nums for c in chunks(chunksize, com_coords))
    com_chain = chain(gen for gen in com_gens)

    return id_arr, batch_arr, coord_arr, count_chain, com_chain

### Short-range gravity.

In [ ]:
# Load files from cell division.
fin_grid = np.load(f'{TEMP_DIR}/fin_grid_{IDname}.npy')
cell_gen = np.load(f'{TEMP_DIR}/cell_gen_{IDname}.npy')
print(fin_grid.shape)

cell_coords = np.squeeze(fin_grid, axis=1)

print('cell and gen shape:', cell_coords.shape, cell_gen.shape)
cells = len(cell_coords)

# Calculate available memory per core.
mem_so_far = (psutil.virtual_memory().used - OS_MEM)/MB_UNIT
mem_left = 224*1e3 - mem_so_far
core_mem_MB = mem_left / psutil.cpu_count()
print('Per core mem:', core_mem_MB)

# Determine short-range chuncksize based on available memory and cells.
chunksize_sr = chunksize_short_range(
    cells, len(DM_raw), PRE.DM_LIM*SHELL_MULTIPLIERS[-1], core_mem_MB
)

# Split workload into batches (if necessary).
batch_arr, cell_chunks, cgen_chunks = batch_generators_short_range(
    cell_coords, cell_gen, chunksize_sr
)


# Memory profile the short-range gravity function.
# in1 = next(cell_chunks)
# in2 = next(cgen_chunks)
# in3 = batch_arr[0]

# cell_gravity_short_range(
#     in1, in2, snap_GRID_L,
#     DM_pos, PRE.DM_LIM, PRE.DM_SIM_MASS, PRE.SMOOTH_L, 
#     TEMP_DIR, in3
# )

with ProcessPoolExecutor(128) as ex:
    ex.map(
        cell_gravity_short_range, 
        cell_chunks, cgen_chunks, repeat(snap_GRID_L), repeat(DM_pos), 
        repeat(PRE.DM_LIM), repeat(PRE.DM_SIM_MASS), repeat(PRE.SMOOTH_L), 
        repeat(TEMP_DIR), batch_arr
    )

In [ ]:
dPsi_short_range_batches = [
    np.load(f'{TEMP_DIR}/batch{b}_short_range.npy') for b in batch_arr
]
dPsi_short_range = np.array(
    list(chain.from_iterable(dPsi_short_range_batches))
)
np.save(
    f'{TEMP_DIR}/dPsi_short_range_{IDname}.npy', 
    dPsi_short_range
)

print(dPsi_short_range.shape)

### Long-range gravity.

In [ ]:
# Load files from cell division.
fin_grid = np.load(f'{TEMP_DIR}/fin_grid_{IDname}.npy')
DM_count = np.load(f'{TEMP_DIR}/DM_count_{IDname}.npy')
cell_com = np.load(f'{TEMP_DIR}/cell_com_{IDname}.npy')

new_grid = np.squeeze(fin_grid, axis=1)

print('SHAPES:', new_grid.shape, DM_count.shape, cell_com.shape)
cells = len(new_grid)

# Calculate available memory per core.
mem_so_far = (psutil.virtual_memory().used - OS_MEM)/MB_UNIT
mem_left = 224*1e3 - mem_so_far
core_mem_MB = mem_left / psutil.cpu_count()

# Determine long-range chuncksize based on available memory and cells.
chunksize_lr = chunksize_long_range(cells, core_mem_MB)

# Split workload into batches (if necessary).
ids, batches, coords, count_chain, com_chain = batch_generators_long_range(
    new_grid, cell_com, DM_count, chunksize_lr
)


# Memory profile the long-range gravity function.
# in1 = ids[0]
# in2 = batches[0]
# in3 = coords[0]
# in4 = next(count_chain)
# in4 = list(count_chain)[0]
# in5 = next(com_chain)
# in5 = list(com_chain)[0]

# cell_gravity_long_range(
#     in1,in2,in3,in4,in5,PRE.DM_SIM_MASS,PRE.SMOOTH_L,TEMP_DIR
# )

# Pass arrays and chains of generators to multiprocessing routine.
with ProcessPoolExecutor(128) as ex:
    ex.map(
        cell_gravity_long_range, ids, batches, 
        coords, count_chain, com_chain,
        repeat(PRE.DM_SIM_MASS), repeat(PRE.SMOOTH_L), repeat(TEMP_DIR)
    )

# with Pool(128) as pool:
#     pool.starmap(
#         cell_gravity_long_range, zip(
#         ids, batches, 
#         coords, count_chain, com_chain,
#         repeat(PRE.DM_SIM_MASS), repeat(PRE.SMOOTH_L), repeat(TEMP_DIR))
#     )

In [ ]:
def load_dPsi(c_id, batches, out_dir):
    
    # Load all batches for current cell.
    dPsi_raw = np.array(
        [np.load(f'{out_dir}/cell{c_id}_batch{b}_long_range.npy') for b in batches]
    )

    # Delete entry, which corresponds to current cell. That "in-cell" gravity 
    # is calculated more precisely with the short-range gravity function.
    dPsi_cell_i = np.sum(np.delete(dPsi_raw, c_id-1, axis=0), axis=0)
    np.save(f'{out_dir}/cell{c_id}_long_range.npy', dPsi_cell_i)    


# Pass arrays and chains of generators to multiprocessing routine.
with ProcessPoolExecutor(PRE.PRE_CPUs) as ex:
    ex.map(load_dPsi, ids, repeat(batches))

# Save long-range gravity (a [x,y,z] array) for all cells.
dPsi_cells = [np.load(f'{TEMP_DIR}/cell{c}_long_range.npy') for c in ids]
np.save(f'{TEMP_DIR}/dPsi_grid_{IDname}.npy', np.array(dPsi_cells))

print(np.array(dPsi_cells).shape)